In [12]:
%%capture
!pip install -r ../requirements.txt

In [13]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add it to sys.path if not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [14]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [15]:
from notion_client import Client
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from twilio.rest import Client as TwilioClient
from typing import TypedDict
import os
from tools.notion_fetch_agent_tools import fetch_latest_notion_journaling_entry
from typing import TypedDict
from langgraph.graph import END, StateGraph

In [16]:
# TEMPLATE ENV VARIABLES
GRAPH_ENTRY_QUERY = "Send me bible scripture that will help me with how i want to grow from my latest journaling in Notion."

In [17]:
llm = ChatOpenAI(model="gpt-4o")

In [18]:
class State(TypedDict):
    query: str
    notion_data_action: str # TEMPLATE VAR
    notion_next_node_instructions: str # TEMPLATE VAR
    notion_journal_growth_summary: str # TEMPLATE VAR
    message: str # TEMPLATE VAR

## Agent 1 - Notion

In each agent (agents/), we can have the this code as attributes/methods

In [19]:
from langchain.agents import initialize_agent

In [20]:
tools = [fetch_latest_notion_journaling_entry]

In [21]:
notion_prompt_template = PromptTemplate(
    input_variables=["notion_data_action"],
    template="You are an assistant responsible for simply making a call to the notion client API, and then doing the following instructions on the returned data: {notion_data_action}"
)

In [22]:
notion_agent = initialize_agent(
    tools=tools,
    llm=llm,
    handle_parsing_errors=True
)

/var/folders/1p/k8wcs51j0bndgtngv7p3gbsr0000gn/T/ipykernel_92160/1288154412.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  notion_agent = initialize_agent(


In [23]:
result = notion_agent.invoke(notion_prompt_template.format(notion_data_action="From this journaling, extract out the ways the user desires to grow."))

In [24]:
result

{'input': 'You are an assistant responsible for simply making a call to the notion client API, and then doing the following instructions on the returned data: From this journaling, extract out the ways the user desires to grow.',
 'output': 'The user desires to grow in spiritual and personal development, leadership, intrinsic motivation, perseverance, meaningful relationships, and through gratitude and reflective practices.'}

In [25]:
def notion_node(state: State) -> State:
    notion_data_action = state.get("notion_data_action")
    
    prompt = notion_prompt_template.format(notion_data_action=notion_data_action)
    result = notion_agent.invoke(prompt)
    return {"notion_journal_growth_summary": result.get('output')}

## Agent 2 - Scripture (generation)

In [26]:
generation_agent_instructions = """
You are a generalist assistant reponsible for performing a simple query. The 
instructions are:

{instructions}
"""

In [27]:
generation_agent_prompt_template = PromptTemplate(
    # TEMPLATE VAR (output of notion node or other things)
    input_variables=["instructions"],
    # TEMPLATE VAR
    template=generation_agent_instructions
)

In [28]:
generation_agent_llm_chain = LLMChain(llm=llm, prompt=generation_agent_prompt_template)

/var/folders/1p/k8wcs51j0bndgtngv7p3gbsr0000gn/T/ipykernel_92160/2322471595.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  generation_agent_llm_chain = LLMChain(llm=llm, prompt=generation_agent_prompt_template)


In [29]:
# Testing
x = "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: {notion_journal_growth_summary}"
x = generation_agent_prompt_template.format(instructions=x).format(notion_journal_growth_summary="The user desires to grow spiritually, as a leader, by embracing intrinsic motivation, strengthening relationships, maintaining joy and gratitude, and through personal accountability and a growth mindset.")
generation_agent_llm_chain.run(x)

/var/folders/1p/k8wcs51j0bndgtngv7p3gbsr0000gn/T/ipykernel_92160/3269216885.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  generation_agent_llm_chain.run(x)


'A scripture that aligns well with your goals for personal growth is Philippians 4:13: "I can do all things through Christ who strengthens me." This verse encourages intrinsic motivation and reliance on spiritual strength, which can bolster your leadership skills, reinforce your relationships, and support your pursuit of joy, gratitude, personal accountability, and a growth mindset.'

In [30]:
def generation_node(state: State) -> State:
    # Output from previous node
    # TEMPLATE VAR
    notion_journal_growth_summary = state.get("notion_journal_growth_summary")
    # TEMPLATE VAR
    notion_next_node_instructions = state.get("notion_next_node_instructions") + notion_journal_growth_summary
    
    prompt = generation_agent_prompt_template.format(instructions=notion_next_node_instructions)
    output = generation_agent_llm_chain.run(prompt)

    return {"message": output}

## Agent 3 - Twilio

In [31]:
from langchain.tools import tool

TO_NUM = "xyz" # TEMPLATE VAR
FROM_NUM = "xyz" # ENV VAR for the job (set on Twilio console)

@tool
def text_user(msg: str):
    """Tool to text the user via Twilio."""
    # account_sid = os.getenv("TWILIO_ACCOUNT_SID") # TODO: get these
    # auth_token = os.getenv("TWILIO_AUTH_TOKEN")
    # from_number = os.getenv("TWILIO_FROM_NUMBER")
    # client = Client(account_sid, auth_token)
    # sms = client.messages.create(
    #     body=msg,
    #     from_=from_num,
    #     to=to_num
    # )
    #return sms.sid
    print(f"Sent {msg}")

In [32]:
twilio_prompt_template = PromptTemplate(
    # TEMPLATE VAR (output of notion node or other things)
    input_variables=["message"],
    # TEMPLATE VAR
    template="Your sole purpose is to send this text message to the user: \"{message}\""
)

In [33]:
tools = [text_user]
twilio_agent = initialize_agent(
    tools=tools,
    llm=llm,
    handle_parsing_errors=True
)

In [34]:
def twilio_node(state: State) -> State:
    output = twilio_agent.invoke(twilio_prompt_template.format(message=state.get("message")))
    return {}

## Agent 4 - Reasoning (with R1)

In [35]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

In [11]:
# Initialize the ChatDeepSeek model
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Create a prompt template
prompt = PromptTemplate(
    input_variables=["instructions"],
    template="""
You are an agent with reasoning capabilities, reponsible for the following:

{instructions}
"""
)

# Create the chain using the runnables API (prompt | llm | parser)
chain = prompt | llm | StrOutputParser()

# Run the chain
response = chain.invoke({"instructions": "What is the capital of France?"})
print(response)

The capital of France is **Paris**. This city is renowned for its cultural landmarks, including the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. Paris has been the political and administrative center of France for centuries, housing key institutions like the French government, the President's residence (Élysée Palace), and the National Assembly.


In [45]:
def reasoning_node(state: State) -> State:
    # Output from previous node
    # TEMPLATE VAR
    notion_journal_growth_summary = state.get("notion_journal_growth_summary")
    # TEMPLATE VAR
    notion_next_node_instructions = state.get("notion_next_node_instructions") + notion_journal_growth_summary

    print("Reasoning...")
    response = chain.invoke({"instructions": notion_next_node_instructions})
    return {"message": response}


## Graph

In [46]:
# Initialize the graph with the state schema
graph = StateGraph(State)

# Add nodes
# TEMPLATE VAR - We can loop through the nodes in the actual app
graph.add_node("notion_node", notion_node)
#graph.add_node("generation_node", generation_node)
graph.add_node("reasoning_node", reasoning_node) # For simplicity, lets have a given AI job use either generation or reasoning, not both (hence, both set "message" attribute)
graph.add_node("twilio_node", twilio_node)

# Set the entry point
# TEMPLATE VAR
graph.set_entry_point("notion_node")

# Define sequential edges
# TEMPLATE VAR -Also loop through this
graph.add_edge("notion_node", "reasoning_node")
graph.add_edge("reasoning_node", "twilio_node")
graph.add_edge("twilio_node", END)

compiled_graph = graph.compile()

In [47]:
# This initial state is going to contain the injected variables from the orchestrator
initial_state = {
    "query": "asdas",
    "notion_data_action": "From this journaling, extract out the primary way the user desires to grow in his/her life.",
    "notion_next_node_instructions": "Based on this growth summary of a user's journal page, provide a single Bible scripture/verse that will help them with personal growth: "
}
try:
    result = compiled_graph.invoke(initial_state)
    print("Final state:", result)
except Exception as e:
    print(f"Graph-level error: {e}")

Reasoning...
Sent **Bible Verse:**  \n*\"Each of you should use whatever gift you have received to serve others, as faithful stewards of God’s grace in its various forms.\"*  \n**—1 Peter 4:10 (NIV)**  \n\n**Reasoning:**  \n- **Servant Leadership & Using Gifts:** Directly emphasizes stewarding God-given gifts to serve others, aligning with servant leadership.  \n- **Supportive Community:** Serving others fosters connection and communal support.  \n- **Deepening Faith:** Calls believers to act as \"faithful stewards,\" anchoring service in trust and gratitude for God’s grace.  \n- **Intrinsic Motivation & Perseverance:** Serving as a response to divine grace cultivates purpose-driven endurance, reinforcing perseverance rooted in faith rather than external rewards.  \n\nThis verse holistically addresses the user’s growth goals while grounding their journey in biblical stewardship and love.
Final state: {'query': 'asdas', 'notion_data_action': 'From this journaling, extract out the primar